In [2]:
import numpy as np
from typing import List
from tqdm import tqdm
from typing import Tuple

## Лабораторная работа 12. Матчасть DL

1. Класс Neuron, имеющий вектор весов self._weigths

2. Два метода класса Neuron: forward(x), backward(x, loss) - реализующих прямой и обратный проход по нейронной сети. Метод forward должен реализовывать логику работу нейрона: умножение входа на вес self._weigths, сложение и функцию активации сигмоиду. Метод backward должен реализовывать взятие производной от сигмоиды и используя состояние нейрона обновить его веса.

3. Реализовать с помощью класса Neuron нейронную сеть с архитектурой из трёх нейронов, предложенную в статье.

4. Для красоты обернуть в класс Model с методами forward и backward, реализующими правильное взаимодействие нейронов на прямом и обратном проходах.

5. Реализовать тренировочный цикл следующего вида:
```
цикл (обучающие данные):
      y = model.forward(x)
      err = loss(y, label)
      model.backward(x, err)
```

In [3]:
def sigmoid(x):
  return 1/(1 + np.exp(-x))

def d_sigmoid(x):
    return x * (1 - x)

In [ ]:
class Neuron:
  def __init__ (self, weights : list[int], bias : float):
    self._weights = weights
    self._bias = bias

  def forward(self, x: Tuple[int, int]) -> float:
    """
    Выполняет прямое распространение для входных данных x.

    params:
    x (list): Входные данные в виде списка из двух значений (входные данные для XOR).

    return:
    float: Выход нейрона после применения функции активации (функция сигомиды).
    """
    x = np.array(x)
    sum = np.dot(x, self._weights.T) + self._bias

    return sigmoid(sum)


  def backward(self, output : Tuple [int, int], error : float, learning_rate : float) -> float :


    out_sigmoid = self.forward(output)
    out_d = d_sigmoid(out_sigmoid)

    delta = error * out_d # градиент

    self._weights -= np.dot(output, delta * learning_rate)
    self._bias -= delta * learning_rate

    return delta * self._weights # градиент по весам  слоя


In [5]:
class Model:
    def __init__(self, learning_rate=0.01):
        self._hidden_bias = np.random.uniform(size=(2,))
        self._output_bias = np.random.uniform(size=(1,))
        self._hidden_neurons = [Neuron(np.random.uniform(-1, 1, 2), np.random.uniform(-1, 1)) for _ in range(2)]
        self._output_neuron = Neuron(np.random.uniform(-1, 1, 2), np.random.uniform(-1, 1))
        self._learning_rate = learning_rate

    def forward(self, x: Tuple[int, int]) ->  float:
        hidden_outputs = tuple(neuron.forward(x) for neuron in self._hidden_neurons)
        predict = self._output_neuron.forward(hidden_outputs)
        return predict

    def backward(self, input: Tuple[int, int], error: float):
        hidden_outputs = tuple(neuron.forward(input) for neuron in self._hidden_neurons)
        delta = self._output_neuron.backward(hidden_outputs, error, self._learning_rate)
        for i, neuron in enumerate(self._hidden_neurons):
            neuron.backward(input, delta[i], self._learning_rate)



In [6]:
 def loss(predict: float, label: float) -> float:
    return 1/2*(label - predict)**2

In [11]:
inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
expected_outputs = np.array([0, 1, 1, 0])


model = Model(learning_rate=0.1)

los_val = 0
count = 0

epochs = 10000
for epoch in tqdm(range(epochs)):
    los_val = 0
    for x, y_true in zip(inputs, expected_outputs):
        y = model.forward(x)
        err = y - y_true
        los_val+=err
        model.backward(x, err)

    count+=1
    if (count == 1000):
      count = 0
      print(f"Epoch {epoch}, Loss: {los_val}")

# Тестирование
for x in inputs:
    y_pred = model.forward(x)
    print(f"Input: {x}, Predicted Output: {y_pred}")


 18%|█▊        | 1797/10000 [00:00<00:01, 4431.85it/s]

Epoch 999, Loss: 0.001784683439491619


 27%|██▋       | 2700/10000 [00:00<00:01, 4345.32it/s]

Epoch 1999, Loss: 0.005356640808145463


 36%|███▌      | 3572/10000 [00:00<00:01, 4339.95it/s]

...

Epoch 7999, Loss: 0.011421745994894636


 97%|█████████▋| 9717/10000 [00:02<00:00, 4252.96it/s]

Epoch 8999, Loss: 0.010178364768664017


100%|██████████| 10000/10000 [00:02<00:00, 4282.42it/s]

Epoch 9999, Loss: 0.009258872032153219
Input: [0 0], Predicted Output: 0.05729060743382826
Input: [0 1], Predicted Output: 0.9512132513438234
Input: [1 0], Predicted Output: 0.9500871615925586
Input: [1 1], Predicted Output: 0.050679856524528714
